In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import nltk
import re
from tensorflow.keras.layers import GRU, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.python.keras import models
from tensorflow.python.keras import initializers
from tensorflow.python.keras import regularizers

from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import SeparableConv1D
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Activation, Dense, Bidirectional

In [2]:
!pip install datasets;

     |████████████████████████████████| 269 kB 4.0 MB/s 
     |████████████████████████████████| 1.3 MB 30.8 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 119 kB 57.1 MB/s 
     |████████████████████████████████| 243 kB 49.8 MB/s 
     |████████████████████████████████| 142 kB 49.4 MB/s 
     |████████████████████████████████| 294 kB 51.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0


In [3]:
from datasets import load_dataset
dataset = load_dataset("offenseval2020_tr")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset offenseval2020_tr downloaded and prepared to /root/.cache/huggingface/datasets/offenseval2020_tr/offenseval2020-turkish/1.0.0/cfae6cd084e04bff3b4fb9e4eeb072e77aae596bfd1f4b919bb06e4e2656a583. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [142]:
data = []
labels = []

In [143]:
for i in dataset['train']:
  data.append(i['tweet'])
  labels.append(i['subtask_a'])

In [144]:
data2 = pd.read_csv('/content/bad.txt',sep='\n')

In [145]:
data2 = np.array(data2)

In [146]:
for i in range(len(np.array(data2))):
  data.append(data2[i][0])
  labels.append(1)

In [147]:
labels = np.array(labels)

In [148]:
data = list(data)

In [149]:
def take_data_to_shower(tweet):
    noises = ['URL', '@USER']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

In [150]:
data = map(lambda x: take_data_to_shower(x),data)

In [151]:
data = map(lambda x: re.sub(r"[-()\"#/@;:<>{}+=~|.!?,]", '', x), data )
data = map(lambda x: x.lower(),data)
data = map(lambda x: x.strip(),data) 

In [152]:
data = list(data)

In [153]:
final_data = []
for i in data:
  final_data.append(i.replace('user',''))

In [154]:
data = final_data
del final_data

In [155]:
nltk.download('stopwords')
stop_word_list = nltk.corpus.stopwords.words('turkish')

def token(values):
    filtered_words = [word for word in values.split() if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc


data = list(map(lambda x: token(x),data)) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [156]:
X_train, X_val, y_train, y_val =train_test_split(data,labels,test_size=0.2, stratify= labels)

In [157]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text


TOP_K = 20000
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):

    global tokenizer
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH
        

    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    
    return x_train, x_val, tokenizer.word_index

In [158]:
def _get_last_layer_units_and_activation(num_classes):
  
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

In [159]:
def sepcnn_model(blocks,
                 filters,
                 kernel_size,
                 embedding_dim,
                 dropout_rate,
                 pool_size,
                 input_shape,
                 num_classes,
                 num_features,
                 use_pretrained_embedding=False,
                 is_embedding_trainable=False,
                 embedding_matrix=None):
  
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    
    if use_pretrained_embedding:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0],
                            weights=[embedding_matrix],
                            trainable=is_embedding_trainable))
    else:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0]))

    for _ in range(blocks-1):
        model.add(Dropout(rate=dropout_rate))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(MaxPooling1D(pool_size=pool_size))

    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(op_units, activation=op_activation))
    return model

In [160]:
def get_num_classes(labels):
    num_classes = max(labels) + 1
    missing_classes = [i for i in range(num_classes) if i not in labels]
    if len(missing_classes):
        raise ValueError('Missing samples with label value(s) '
                         '{missing_classes}. Please make sure you have '
                         'at least one sample for every label value '
                         'in the range(0, {max_class})'.format(
                            missing_classes=missing_classes,
                            max_class=num_classes - 1))

    if num_classes <= 1:
        raise ValueError('Invalid number of labels: {num_classes}.'
                         'Please make sure there are at least two classes '
                         'of samples'.format(num_classes=num_classes))
    return num_classes

In [161]:
def train_sequence_model(X_train, X_val, y_train, y_val,
                         learning_rate=5e-3,
                         epochs=1000,
                         batch_size=64,
                         blocks=2,
                         filters=64,
                         dropout_rate=0.4,
                         embedding_dim=100,
                         kernel_size=3,
                         pool_size=3):
  
    (train_texts, train_labels), (val_texts, val_labels) = (X_train,y_train),(X_val,y_val)

    
    num_classes = get_num_classes(train_labels)
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    
    num_features = min(len(word_index) + 1, TOP_K)

    global model
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features)

    
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer='adam', loss=loss, metrics=['acc'])

    
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20)]

    
    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks = [callbacks],
            validation_data=(x_val, val_labels),
            verbose=2,  
            batch_size=batch_size)

    
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    
    #model.save('rotten_tomatoes_sepcnn_model.h5')
    return history['val_acc'][-1], history['val_loss'][-1]

In [162]:
train_sequence_model(X_train, X_val, y_train, y_val)

Epoch 1/1000
406/406 - 6s - loss: 0.5283 - acc: 0.7894 - val_loss: 0.5146 - val_acc: 0.7896
Epoch 2/1000
406/406 - 4s - loss: 0.4776 - acc: 0.8031 - val_loss: 0.4281 - val_acc: 0.8267
Epoch 3/1000
406/406 - 4s - loss: 0.3580 - acc: 0.8584 - val_loss: 0.4046 - val_acc: 0.8362
Epoch 4/1000
406/406 - 4s - loss: 0.2740 - acc: 0.8940 - val_loss: 0.4373 - val_acc: 0.8327
Epoch 5/1000
406/406 - 4s - loss: 0.2058 - acc: 0.9213 - val_loss: 0.4941 - val_acc: 0.8251
Epoch 6/1000
406/406 - 4s - loss: 0.1555 - acc: 0.9396 - val_loss: 0.6294 - val_acc: 0.8210
Epoch 7/1000
406/406 - 4s - loss: 0.1212 - acc: 0.9514 - val_loss: 0.7362 - val_acc: 0.8040
Epoch 8/1000
406/406 - 4s - loss: 0.0980 - acc: 0.9604 - val_loss: 0.9011 - val_acc: 0.8056
Epoch 9/1000
406/406 - 4s - loss: 0.0777 - acc: 0.9714 - val_loss: 0.9521 - val_acc: 0.8077
Epoch 10/1000
406/406 - 4s - loss: 0.0687 - acc: 0.9762 - val_loss: 1.0311 - val_acc: 0.8107
Epoch 11/1000
406/406 - 5s - loss: 0.0542 - acc: 0.9810 - val_loss: 1.2562 - va

(0.8012632727622986, 2.2206058502197266)

In [163]:
#model = tf.keras.models.load_model('/content/rotten_tomatoes_sepcnn_model.h5')

## Predict

In [164]:
max_length = len(max(X_train, key=len))

In [207]:
sentence = 'karaktersiz'
#sentence = 'nasılsın'

test = tokenizer.texts_to_sequences([sentence])
print(test)
paddedSequence = sequence.pad_sequences(test, maxlen=max_length)

[[2544]]


In [208]:
p = model.predict(np.expand_dims(paddedSequence[0], axis=0))[0]

In [209]:
if p > 0.70:
  print('dava')
else :
  print('normal')

dava


In [210]:
p

array([0.97994256], dtype=float32)

In [211]:
print(model.predict_classes(paddedSequence));

[[1]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
